In [1]:
#1 # Import necessary package
import polars as pl

In [3]:
#2 Load the bike trips and weather data into Polars DataFrames
import polars as pl

trips = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv",
    null_values="NA"
)

weather = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv",
    null_values=["NA", ""]
)

# Ensure the 'date' column is correctly parsed as Polars' Date type
trips = trips.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")
)

weather = weather.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")
)

# (Optional) Quick sanity check: print record counts
print("Trips rows:", trips.height)
print("Weather rows:", weather.height)

Trips rows: 78704
Weather rows: 733


In [4]:
#3 # Join the trips and weather DataFrames on the 'date' column
trips_weather = trips.join(weather, on="date", how="inner")

In [6]:
#5 # Calculate average duration for trips on days with a weather event
avg_bad_weather = (
    trips_weather
    .filter(pl.col("events").is_not_null())  # keep only rows where an event is present
    .select(
        pl.col("duration")  # specify the trip duration column
        .mean()  # compute the average
        .alias("avg_duration")  # name it as 'avg_duration'
    )
)

In [7]:
#6 import polars as pl

# Load the baseball datasets
Batting = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv",
    null_values="NA"
)

People = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv",
    null_values="NA"
)

HallOfFame = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv",
    null_values="NA"
)

# Join Batting and People DataFrames on playerID
stats_w_bio = Batting.join(People, on="playerID", how="inner")

In [8]:
#7 # Calculate total home runs by players born in Florida (FL)
florida_hr = (
    stats_w_bio
    .filter(pl.col("birthState") == "FL")       # Keep only Florida-born players
    .select(pl.col("HR").sum().alias("total_hr"))  # Sum the 'HR' column, alias it as 'total_hr'
)

In [9]:
#8 # Clean the HallOfFame DataFrame to identify Hall-of-Fame players uniquely
hall_inducted = (
    HallOfFame
    .filter(
        (pl.col("inducted") == "Y") & (pl.col("category") == "Player")
    )                           # Keep only rows for inducted players in the 'Player' category
    .unique(subset=["playerID"], keep="first")  # Ensure each playerID appears only once
)

In [11]:
#9 import polars as pl

# Load the datasets
Batting = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv",
    null_values="NA"
)

HallOfFame = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv",
    null_values="NA"
)

# Clean the HallOfFame DataFrame to identify Hall-of-Fame players uniquely
hall_inducted = (
    HallOfFame
    .filter(
        (pl.col("inducted") == "Y") & (pl.col("category") == "Player")
    )                           # Keep only rows for inducted players in the 'Player' category
    .unique(subset=["playerID"], keep="first")  # Ensure each playerID appears only once
)

# Join Batting and hall_inducted DataFrames on playerID and calculate average career hits
hof_hits = (
    Batting
    .join(hall_inducted, on="playerID", how="inner")  # Join on playerID
    .group_by("playerID")  # Group by playerID to aggregate per player
    .agg(pl.col("H").sum().alias("total_hits"))  # Sum hits per player
    .select(pl.col("total_hits").mean().alias("average_hits"))  # Calculate mean of total hits
)

print(hof_hits)

shape: (1, 1)
┌──────────────┐
│ average_hits │
│ ---          │
│ f64          │
╞══════════════╡
│ 1717.29918   │
└──────────────┘
